In [1]:
import pandas as pd
import numpy as np
import glob
import time                
import calendar
import datetime
import os
import difflib
import re
pd.options.mode.chained_assignment = None

In [3]:
def carga_df(dir_resultados, dir_tabla):
    df_list = []
    for data_file in sorted(glob.glob(f'{dir_resultados}/*')):
        df = pd.read_excel(data_file, usecols=lambda x: 'Unnamed' not in x)
        df_list.append(df)
    df_resultado = pd.concat(df_list, axis=0, ignore_index=True)
    regex_fecha = re.compile('(\d+\s\w+\s\d+)')
    df_resultado['Fecha'] = df_resultado['Fecha'].str.extract(regex_fecha)
    df_list.clear()

    for data_file in sorted(glob.glob(f'{dir_tabla}/*')):
        df = pd.read_excel(data_file, usecols=lambda x: 'Unnamed' not in x)
        df_list.append(df)
    df_tabla = pd.concat(df_list, axis=0, ignore_index=True)
    return df_resultado, df_tabla

In [4]:
def comprobar_equipos(df_resultado, df_tabla):
    if set(df_resultado.Equipo_Local) != set(df_tabla.Equipo):
        diff_res = sorted(set(df_resultado.Equipo_Local) - set(df_tabla.Equipo))
        diff_tab = sorted(set(df_tabla.Equipo) - set(df_resultado.Equipo_Local))
        list_remove = []
        for equipo in diff_tab:
            equipo_sustituir = difflib.get_close_matches(equipo, diff_res)
            if len(equipo_sustituir) == 1:
                list_remove.append(equipo)
                df_tabla['Equipo'] = df_tabla['Equipo'].replace(equipo, equipo_sustituir[0])
                diff_res.remove(equipo_sustituir[0])
        diff_tab = list(filter(lambda x: x not in set(list_remove), diff_tab))

        if diff_res:
            print(f'El dataframe de tabla tiene los equipos\n{diff_res}\nque el dataframe de partidos no tiene')
        if diff_tab:
            print(f'El dataframe de partidos tiene los equipos\n{diff_tab}\nque el dataframe de tabla no tiene')
    return diff_res, diff_tab

In [5]:
def generar_keys(df_res, df_tab):
    df_tab['Key'] = df_tab.apply(lambda x:f"{x['Equipo']}-{x['Jornada']}-{x['Año']}-{x['Liga']}", axis=1)
    df_tab['Key_Año_Liga'] = df_tab.apply(lambda x:f"{x['Año']}-{x['Liga']}", axis=1)

    df_res['Key_Local'] = df_res.apply(lambda x:f"{x['Equipo_Local']}-{x['Jornada']}-{x['Año']}-{x['Liga']}", axis=1)
    df_res['Key_Visitante'] = df_res.apply(lambda x:f"{x['Equipo_Visitante']}-{x['Jornada']}-{x['Año']}-{x['Liga']}", axis=1)
    df_res['Key_Año_Liga'] = df_res.apply(lambda x:f"{x['Año']}-{x['Liga']}", axis=1)

    num_partidos = df_res.loc[:,['Key_Año_Liga','Jornada']].groupby(by=['Key_Año_Liga']).max()
    num_equipos = df_res.loc[:,['Key_Año_Liga','Equipo_Local']].groupby(by=['Key_Año_Liga']).nunique()
    dict_n_partidos = num_partidos.to_dict()['Jornada']
    dict_n_equipos = num_equipos.to_dict()['Equipo_Local']

    mask = num_partidos.values == (2*(num_equipos-1)).values
    keys_aceptables = num_partidos[mask].index
    df_res = df_res[df_res['Key_Año_Liga'].isin(keys_aceptables)]
    df_tab = df_tab[df_tab['Key_Año_Liga'].isin(keys_aceptables)]

    return df_res, df_tab, dict_n_partidos, dict_n_equipos

In [6]:
def extraer_resultado_numerico(resultado):
  regex = resultado.str.extract(r'(\d+)-(\d+)')
  goles_local = regex[0]
  goles_visitante = regex[1]
  resultado_local = goles_local.lt(goles_visitante) * (-1) + goles_local.gt(goles_visitante) * 1
  resultado_visitante = goles_local.lt(goles_visitante) * 1 + goles_local.gt(goles_visitante) * (-1)
  resultado_local_1x2 = goles_local.lt(goles_visitante) * 1 + goles_local.eq(goles_visitante) * 2 + goles_local.gt(goles_visitante) * 3
  return resultado_local, resultado_visitante, resultado_local_1x2

In [7]:
df_resultado, df_tabla = carga_df('../Extract_Data/DDBB_Resultados', '../Extract_Data/DDBB_Tabla')
diff_res, diff_tab = comprobar_equipos(df_resultado, df_tabla)

El dataframe de tabla tiene los equipos
['Alianza Petrolera', 'Deportes Tolima', 'Deportivo Cali', 'Fortaleza', 'Independiente Medellin', 'Junior', 'Millonarios', 'Patriotas Boyacá', 'Rionegro Águilas']
que el dataframe de partidos no tiene


In [8]:
df_resultado_filter = df_resultado[~df_resultado['Equipo_Local'].isin(diff_res)]
df_tabla_filter = df_tabla[~df_tabla['Equipo'].isin(diff_tab)]

In [9]:
Resultado, Tabla, dict_n_partidos, dict_n_equipos = generar_keys(df_resultado_filter, df_tabla_filter)
Resultado['Numero_Participantes'] = Resultado['Key_Año_Liga'].map(dict_n_equipos)
Resultado['Total_Jornadas'] = Resultado['Key_Año_Liga'].map(dict_n_partidos)

In [10]:
columnas_dict = ['Posicion', 'Goles_a_favor', 'Goles_en_contra']
dict_key = Tabla[['Key'] + columnas_dict].set_index('Key').to_dict()

for columna in columnas_dict:
  Resultado[columna + '_Local'] = Resultado['Key_Local'].map(dict_key[columna])
  Resultado[columna + '_Visitante'] = Resultado['Key_Visitante'].map(dict_key[columna])
  Resultado.dropna(subset=[columna + '_Local', columna + '_Visitante'], inplace=True)
  Resultado[[columna + '_Local', columna + '_Visitante']] = Resultado[[columna + '_Local', columna + '_Visitante']].astype('int64')

In [11]:
Resultado['Valor_Numerico_Resultado_Local'], Resultado['Valor_Numerico_Resultado_Visitante'], Resultado['1x2']  = extraer_resultado_numerico(Resultado['Resultado'])

In [13]:
Resultado['Racha_Total_Local'] = None
Resultado['Racha_Total_Visitante'] = None
for equipo in set(Resultado['Equipo_Local']):
  equipo_df = Resultado[(Resultado['Equipo_Local'] == equipo) | (Resultado['Equipo_Visitante'] == equipo)]
  for liga in equipo_df['Liga'].unique():
    for año in equipo_df['Año'].unique():
      subset = equipo_df[(equipo_df['Año'] == año) & (equipo_df['Liga'] == liga)].sort_values(by=['Jornada'])[['Equipo_Local', 'Equipo_Visitante', 'Valor_Numerico_Resultado_Local', 'Valor_Numerico_Resultado_Visitante', 'Racha_Total_Local', 'Racha_Total_Visitante']]
      racha = (subset['Equipo_Local'] == equipo).multiply(subset['Valor_Numerico_Resultado_Local']) + (subset['Equipo_Visitante'] == equipo).multiply(subset['Valor_Numerico_Resultado_Visitante'])
      racha_valor = racha.shift(1, fill_value=0) + 0.5 * racha.shift(2, fill_value=0) + 0.2 * racha.shift(3, fill_value=0)
      racha_column_local = pd.Series(racha_valor[subset['Equipo_Local'] == equipo], name = 'Racha_Total_Local')
      racha_column_visitante = pd.Series(racha_valor[subset['Equipo_Visitante'] == equipo], name = 'Racha_Total_Visitante')
      Resultado.update(racha_column_local)
      Resultado.update(racha_column_visitante)